In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install bitarray
!pip install keras
!pip install scikit-learn
!pip install mmh3
!pip install keras==2.1.3

In [0]:
from sklearn import datasets
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import numpy as np

In [0]:
# X.astype('float').tofile('/content/drive/My Drive/DeepBloom-master/data/K-lines-dataset-points.dat')
# y.tofile('/content/drive/My Drive/DeepBloom-master/data/K-lines-dataset-label.dat')
X = np.fromfile('/content/drive/My Drive/DeepBloom-master/data/K-lines-dataset-points.dat').reshape(-1,2)
y = np.fromfile('/content/drive/My Drive/DeepBloom-master/data/K-lines-dataset-label.dat',dtype=int)

In [0]:
plt.plot(X[np.where(y==0),0], X[np.where(y==0),1], '.', color = 'red')
plt.plot(X[np.where(y==1),0], X[np.where(y==1),1], '.', color = 'yellow')
legend_elements = [Line2D([0], [0], color="white",markerfacecolor='yellow',marker='o', label='1'),
            Line2D([0], [0], color="white",markerfacecolor='red',marker='o', label='0')]
plt.title('moon-like-dataset with 10000 points')
plt.legend(handles=legend_elements, loc='upper right')
plt.savefig('/content/drive/My Drive/DeepBloom-master/data/moon-like-dataset with 10000 points.png')



In [0]:
from keras.models import Sequential, load_model
from keras.models import Model
from keras.layers import Dense, Activation, Embedding, Flatten
from keras.layers import LSTM, Input, GRU
from keras import optimizers
import numpy as np
from keras import backend as K
from keras.regularizers import l2
from keras import initializers
from keras.utils.generic_utils import get_custom_objects

initial_learning_rate = 1e-2
batch_size = 64
weight_decay = 0.0005
decay = 0.0001

def focal_loss(alpha_p,alpha_n,gama):

    def loss(y_true,y_pred):
        y_p = 1-y_pred+y_true*(2*y_pred-1)
        alpha = alpha_p*y_true+alpha_n*(1-y_true)
        return -K.mean(alpha*K.pow(1-y_p,gama)*K.log(y_p))
    return loss

def focal_loss_plus2(alpha_p,alpha_n,gama):

    def loss(y_true,y_pred):
        y_p = 1-y_pred+y_true*(2*y_pred-1)
        alpha = alpha_p*y_true+alpha_n*(1-y_true)
        k = K.pow(1-y_p,gama)*y_true+(1-y_true)
        return -K.mean(alpha*k*K.log(y_p))
    return loss

def my_model(x):
  x = Dense(2, activation='relu', name='hidden',kernel_regularizer=l2(weight_decay))(x)
  x = Dense(1, activation='sigmoid', name='output',kernel_regularizer=l2(weight_decay))(x)
  return x


def run(train_X,train_y,epochs,alpha_p,alpha_n,gama):
  inputs = Input((2,))
  out = my_model(inputs)
  model = Model(inputs=inputs, outputs=out)
  model.summary()
  optimizer = optimizers.Adam(lr=initial_learning_rate, decay=decay)
  #optim = optimizers.SGD(lr=initial_learning_rate, momentum=0.9)
  model.compile(optimizer=optimizer,loss=focal_loss_plus1(alpha_p,alpha_n,gama),metrics=['accuracy'])
  model.fit(train_X, train_y, batch_size=5000, epochs=epochs)
  #reduced_model = Model(inputs=model.input, outputs=model.get_layer('hidden').output)
  return model



In [0]:
def my_visualize(X,y,a,epoch,layer,loss,fnr,fpr,time,alpha_p,alpha_n,gama,threshold):
  plt.figure()
  plt.clf()
  b = np.where(y==0)[0]
  c = np.where(y==1)[0]
  d = set(a)&set(c)
  d = list(d)
  e = set(a)&set(b)
  e = list(e)
  plt.plot(X[b,0], X[b,1], '.', color = 'red')
  plt.plot(X[c,0], X[c,1], '.', color = 'yellow')
  plt.plot(X[d,0], X[d,1], '.', color = 'blue')
  plt.plot(X[e,0],X[e,1], '.', color = 'green')
  legend_elements = [Line2D([0], [0], color="white",markerfacecolor='yellow',marker='o', label='1'),
            Line2D([0], [0], color="white",markerfacecolor='red',marker='o', label='0'),
            Line2D([0], [0], color="white",markerfacecolor='green',marker='o', label='f'),
            Line2D([0], [0], color="white",markerfacecolor='blue',marker='o', label='t')]
  plt.title('Ap-{}-an-{}-gama-{}-FNR-{}'.format(alpha_p,alpha_n,gama,fnr)+'-FPR-{:.4f}'.format(fpr)+'-Thr-{:.3f}'.format(threshold))
  plt.legend(handles=legend_elements, loc='upper right')
  plt.savefig('/content/drive/My Drive/DeepBloom-master/data/'+'EX1-Layer-'+layer+'-loss-'+loss+'-time-{}-ap-{}-an-{}-gama-{}-Epoch-{}-FNR-{}-train.png'.format(time,alpha_p,alpha_n,gama,epoch,fnr))
  plt.close()

In [0]:
def draw_histogram(predictions,y,epoch,layer,loss,fnr,fpr,time,alpha_p,alpha_n,gama,threshold):
  plt.figure()
  plt.clf()
  positive = predictions[np.where(y==1)]
  plt.hist(positive, bins = 20)
  plt.title('His-Pos-'+'ap-{}-an-{}-gama-{}-Epoch-{}-FNR-{}'.format(alpha_p,alpha_n,gama,epoch,fnr)+'-FPR-{:.4f}'.format(fpr)+'-Thr-{:.3f}'.format(threshold))
  plt.savefig('/content/drive/My Drive/DeepBloom-master/data/'+'EX1-Histogram-Positive-Layer-'+layer+'-loss-'+loss+'-time-{}-ap-{}-an-{}-gama-{}-Epoch-{}-FNR-{}-train.png'.format(time,alpha_p,alpha_n,gama,epoch, fnr))
  plt.close()
  plt.figure()
  plt.clf()
  negative = predictions[np.where(y==0)]
  plt.hist(negative, bins = 20)
  plt.title('His-Neg-'+'ap-{}-an-{}-gama-{}-Epoch-{}-FNR-{}'.format(alpha_p,alpha_n,gama,epoch,fnr)+'-FPR-{:.4f}'.format(fpr)+'-Thr-{:.3f}'.format(threshold))
  plt.savefig('/content/drive/My Drive/DeepBloom-master/data/'+'EX1-Histogram-Negative-Layer-'+layer+'-loss-'+loss+'-time-{}-ap-{}-an-{}-gama-{}-Epoch-{}-FNR-{}-train.png'.format(time,alpha_p,alpha_n,gama,epoch, fnr))
  plt.close()
  

In [0]:
def my_experiment(X,y,epoch,false_negative,layer,loss,time,alpha_p,alpha_n,gama):
  model_1 = run(X,y,epoch,alpha_p,alpha_n,gama)
  weights = model_1.get_layer('output').get_weights()
  predictions = model_1.predict(X)
  Trues = predictions[np.where(y==0)]
  sort_true = np.sort(Trues,axis=0)
  threshold = sort_true[round((1-false_negative)*len(X)/2)]
  print(threshold)
  threshold_t = -np.log(1.0/threshold-1)
  b = np.where(predictions>threshold)[0]
  loss_len = len(b)-round(false_negative*len(X)/2)
  false_positive = 1-loss_len/(len(X)/2)
  a = np.where(predictions>threshold)[0]
  threshold = np.float(threshold)
  draw_histogram(predictions,y,epoch,layer,loss,false_negative,false_positive,time,alpha_p,alpha_n,gama,threshold)
  my_visualize(X,y,a,epoch,layer,loss,false_negative,false_positive,time,alpha_p,alpha_n,gama,threshold)
  print(false_positive)
  return model_1

In [0]:
false_negative = 0.01
epoch = 100
layer = '2-1'
loss = 'focal+2'
time = 3
alpha_p = 1
alpha_n = 2
gama = 2
model_1 = my_experiment(X,y,epoch,false_negative,layer,loss,time,alpha_p,alpha_n,gama)